In [2]:
# Chapter 22
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as scp
import math

In [3]:
n = 4601
k = 58
colNames = [i for i in range(k)]
data = pd.read_csv('data/spam.txt', delim_whitespace = True, names=colNames)
data

,0,1,2,3,4,5,6,7,8,9,...,48,49,50,51,52,53,54,55,56,57
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.000,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.010,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.000,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4596,0.31,0.00,0.62,0.0,0.00,0.31,0.00,0.00,0.00,0.00,...,0.000,0.232,0.0,0.000,0.000,0.000,1.142,3,88,0
4597,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.000,0.0,0.353,0.000,0.000,1.555,4,14,0
4598,0.30,0.00,0.30,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.102,0.718,0.0,0.000,0.000,0.000,1.404,6,118,0
4599,0.96,0.00,0.00,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.000,0.057,0.0,0.000,0.000,0.000,1.147,5,78,0


In [4]:
Y = data[k-1].to_numpy()
X = data[[i for i in range(k-1)]].to_numpy()

In [ ]:
#LDA

In [ ]:
n1 = Y.sum()
n0 = n-n1
pi1 = n1/n
pi0 = n0/n
pi0

In [ ]:
X0 = X[[Y[i]==0 for i in range(n)]]
X1 = X[[Y[i]==1 for i in range(n)]]
mu0 = np.mean(X0, axis=0)
mu1 = np.mean(X1, axis=0)
S0 = np.mean( np.array([np.outer(X0[i]-mu0, X0[i]-mu0) for i in range(n0)]) , axis=0)
S1 = np.mean( np.array([np.outer(X1[i]-mu1, X1[i]-mu1) for i in range(n1)]) , axis=0)
S = 1/n*(n0*S0 + n1*S1)

In [ ]:
def LDA(X, Y, pi0, pi1, mu0, mu1, S, summarize=False):
    n, k = X.shape
    Sinv = np.linalg.inv(S)
    Yhat = np.zeros(n)
    summary = np.zeros((2, 2),dtype=np.int64)
    for i in range(n):
        delta0 = np.log(pi0) + X[i].dot(Sinv).dot(mu0)-1/2*mu0.dot(Sinv).dot(mu0)
        delta1 = np.log(pi1) + X[i].dot(Sinv).dot(mu1)-1/2*mu1.dot(Sinv).dot(mu1) 
        Yhat[i] = delta0 < delta1
        summary[int(delta0 < delta1)][Y[i]] +=1
    trainingErr = (summary[0][1] + summary[1][0])/n
    if(summarize):
        print(summary)
    return Yhat, trainingErr
            

In [ ]:
Yhat, trainingErr = LDA(X, Y, pi0, pi1, mu0, mu1, S, summarize=True)
print(trainingErr)

In [ ]:
#QDA

In [ ]:
def QDA(X, Y, pi0, pi1, mu0, mu1, S0, S1, summarize=False):
    n, k = X.shape
    Sinv0 = np.linalg.inv(S0)
    Sinv1 = np.linalg.inv(S1)
    Det0 = np.linalg.det(S0)
    Det1 = np.linalg.det(S1)
    Yhat = np.zeros(n)
    summary = np.zeros((2, 2),dtype=np.int64)
    for i in range(n):
        delta0 = np.log(pi0) -1/2*np.log(Det0) -1/2*(X[i]-mu0).dot(Sinv0).dot(X[i]-mu0)
        delta1 = np.log(pi1) -1/2*np.log(Det1) -1/2*(X[i]-mu1).dot(Sinv1).dot(X[i]-mu1)
        Yhat[i] = delta0 < delta1
        summary[int(delta0 < delta1)][Y[i]] +=1
    trainingErr = (summary[0][1] + summary[1][0])/n
    if(summarize):
        print(summary)
    return Yhat, trainingErr

In [ ]:
Yhat, trainingErr=QDA(X, Y, pi0, pi1, mu0, mu1, S0, S1, summarize=True)
print(trainingErr)

In [ ]:
#logistic regression

In [ ]:
def multiLinearFit(X, W, Y):
    n, k = X.shape
    beta = None
    try:
        beta = np.linalg.inv(X.transpose().dot(W).dot(X)).dot(X.transpose()).dot(W).dot(Y)
    except:
        print(X)
    Yhat = X.dot(beta)
    trainingErr = np.sum((Y-Yhat)**2)
    sig = (trainingErr/(n-k))**0.5
    seMatrix = sig**2*np.linalg.inv(X.transpose().dot(X))
    return beta, sig, seMatrix, trainingErr

In [ ]:
def logisticFit(X, Y, maxIter=1000, summarize=False):
    n, k = X.shape
    beta = np.zeros(shape=k)
    
    def logit(x):
        return np.log(x/1-x)
    def logitInv(x):
        return 1/(np.exp(-x)+1)
    
    Z = np.zeros(shape=n)
    p = np.zeros(shape=n)
    seMatrix = None
    for _ in range(maxIter):
        for i in range(n):
            p[i] = logitInv(beta.dot(X[i]))
            Z[i] = beta.dot(X[i]) + (Y[i]-p[i])/(p[i]*(1-p[i]))
        W = np.diag([float(p[i])*(1-p[i]) for i in range(n)])
        beta, sig, seMatrix, trainingErr = multiLinearFit(X, W, Z)
    
    Zhat = X.dot(beta)
    phat = logitInv(Zhat)
    print(beta)
    if summarize:
        summary = np.zeros((2, 2),dtype=np.int64)
        for i in range(n):
            summary[int(phat[i]>0.5)][Y[i]] += 1
        print(summary)
    trainingErrRate = np.mean(np.array([(phat[i]<0.5 and Y[i]==1) or (phat[i]>0.5 and Y[i]==0) for i in range(n)]))       
    return beta, seMatrix, trainingErrRate

In [ ]:
beta, seMatrix, trainingErrRate = logisticFit(X, Y, maxIter=4, summarize=True)
print(trainingErrRate)

In [ ]:
#cross validate LDA and logistic

In [ ]:
V = 5

In [ ]:
m=n//V
indices = np.arange(V*m)
permIndices = np.random.choice(indices, V*m, replace=False)

In [ ]:
Xperm = X2[permIndices]
Yperm = Y[permIndices]

In [ ]:
Xcv = Xperm.copy().reshape( (V, m, k2))
Ycv = Yperm.copy().reshape((V, m))   

In [ ]:
#LDA

In [ ]:
def LDA_train(X, Y):
    n, k = X.shape
    n1 = Y.sum()
    n0 = n-n1
    pi1 = n1/n
    pi0 = n0/n
    X0 = X[[Y[i]==0 for i in range(n)]]
    X1 = X[[Y[i]==1 for i in range(n)]]
    mu0 = np.mean(X0, axis=0)
    mu1 = np.mean(X1, axis=0)
    S0 = np.mean( np.array([np.outer(X0[i]-mu0, X0[i]-mu0) for i in range(n0)]) , axis=0)
    S1 = np.mean( np.array([np.outer(X1[i]-mu1, X1[i]-mu1) for i in range(n1)]) , axis=0)
    S = 1/n*(n0*S0 + n1*S1)
    return pi0, pi1, mu0, mu1, S

In [ ]:
def LDA_test(X, Y, pi0, pi1, mu0, mu1, S, summarize=False):
    n, k = X.shape
    Sinv = np.linalg.inv(S)
    Yhat = np.zeros(n)
    summary = np.zeros((2, 2),dtype=np.int64)
    for i in range(n):
        delta0 = np.log(pi0) + X[i].dot(Sinv).dot(mu0)-1/2*mu0.dot(Sinv).dot(mu0)
        delta1 = np.log(pi1) + X[i].dot(Sinv).dot(mu1)-1/2*mu1.dot(Sinv).dot(mu1) 
        Yhat[i] = delta0 < delta1
        summary[int(delta0 < delta1)][Y[i]] +=1
    testErr = (summary[0][1] + summary[1][0])/n
    if(summarize):
        print(summary)
    return Yhat, testErr

In [ ]:
errs = np.zeros(V)
for v in range(V):
    Xtr = Xperm[ [i < v*m or i >= (v+1)*m for i in range(V*m)] ]
    Ytr = Yperm[ [i < v*m or i >= (v+1)*m for i in range(V*m)] ]
    pi0, pi1, mu0, mu1, S = LDA_train(Xtr, Ytr)
    Yhat, testErr = LDA_test(Xcv[v], Ycv[v], pi0, pi1, mu0, mu1, S)
    errs[v] = testErr
errs.mean() #cross validation err rate for LDA

In [ ]:
errs = np.zeros(V)
for v in range(V):
    Xtr = Xperm[ [i < v*m or i >= (v+1)*m for i in range(V*m)] ]
    Ytr = Yperm[ [i < v*m or i >= (v+1)*m for i in range(V*m)] ]
    pi0, pi1, mu0, mu1, S = LDA_train(Xtr, Ytr)
    Yhat, testErr = LDA_test(Xcv[v], Ycv[v], pi0, pi1, mu0, mu1, S)
    errs[v] = testErr
errs.mean() #cross validation err rate for LDA with reduced number of covariates

In [ ]:
errs

In [ ]:
#logistic regression

In [ ]:
def logistic_train(X, Y, maxIter=1000):
    n, k = X.shape
    beta = np.zeros(shape=k)
    
    def logit(x):
        return np.log(x/1-x)
    def logitInv(x):
        return 1/(np.exp(-x)+1)
    
    Z = np.zeros(shape=n)
    p = np.zeros(shape=n)
    seMatrix = None
    for _ in range(maxIter):
        for i in range(n):
            p[i] = logitInv(beta.dot(X[i]))
            Z[i] = beta.dot(X[i]) + (Y[i]-p[i])/(p[i]*(1-p[i]))
        W = np.diag([float(p[i])*(1-p[i]) for i in range(n)])
        beta, sig, seMatrix, trainingErr = multiLinearFit(X, W, Z)
    return beta, sig, seMatrix

def logistic_test(X, Y, beta, summarize=False):
    n, k = X.shape
    def logit(x):
        return np.log(x/1-x)
    def logitInv(x):
        return 1/(np.exp(-x)+1)
    Zhat = X.dot(beta)
    phat = logitInv(Zhat)
    if summarize:
        summary = np.zeros((2, 2),dtype=np.int64)
        for i in range(n):
            summary[int(phat[i]>0.5)][Y[i]] += 1
        print(summary)
    testErrRate = np.mean(np.array([(phat[i]<0.5 and Y[i]==1) or (phat[i]>0.5 and Y[i]==0) for i in range(n)]))       
    return phat, testErrRate

In [ ]:
errs = np.zeros(V)
for v in range(V):
    Xtr = Xperm[ [i < v*m or i >= (v+1)*m for i in range(V*m)] ]
    Ytr = Yperm[ [i < v*m or i >= (v+1)*m for i in range(V*m)] ]
    beta, sig, seMatrix = logistic_train(Xtr, Ytr, maxIter = 4)
    Yhat, testErr = logistic_test(Xcv[v], Ycv[v], beta)
    errs[v] = testErr
errs.mean()

In [ ]:
#choose best 10 covariates

In [ ]:
def WaldStats(j):
    diff = np.abs(mu0[j]-mu1[j])
    se = (S0[j][j]/n0 + S1[j][j]/n1)**0.5
    return diff/se

k = 57
k2 = 30
W = [WaldStats(j) for j in range(k)]
bestCov = []
for val in sorted(W)[::-1]:
    bestCov.append(W.index(val))

bestCov = bestCov[0:k2]


In [ ]:
X2 = X[:,bestCov]

In [ ]:
pi0t, pi1t, mu0t, mu1t, St = LDA_train(X2, Y)
Yhat, testErr = LDA_test(X2, Y, pi0t, pi1t, mu0t, mu1t, St, summarize=True)
print(testErr)

In [ ]:
beta, sig, seMatrix = logistic_train(X2, Y, maxIter=5)

In [ ]:
phat, testErrRate  = logistic_test(X2, Y, beta, summarize=True)
print(testErrRate)

In [ ]:
#SVM

In [9]:
import sklearn.svm as svm

In [28]:
sz = 2000
indices = np.arange(n)
permIndices = np.random.choice(indices, sz, replace=False)
Xperm = X[permIndices]
Yperm = Y[permIndices]

In [13]:
clf = svm.SVC(kernel="linear")

In [ ]:
clf.fit(Xperm, Yperm)

In [24]:
Yhat = clf.predict(X)

In [27]:
#training error rate
(Yhat != Y).mean()

0.099978265594436